In [1]:
import numpy as np
import pandas as pd

# Cargar Data

In [2]:
path='../SOURCES/' 
path_inp='%sResults2.csv' %(path)
# path_inp='%sResults.csv' %(path)

In [3]:
data = pd.read_csv(path_inp, sep=",", header=0,  dtype={'week': str,'year': str})
# data = data.head(100)
data['time'] = data['year']+"-"+data['week']
data.head(10)

,customer_tag,year,week,profile_id,mccg,turn,size,m2t0d0,m2t0d1,m2t0d2,...,m28t3d0,m28t3d1,m28t3d2,m28t3d3,m28t3d4,m28t3d5,m28t3d6,individual_cluster,collective_cluster,time
0,+++g8j9k+5A=,2016,27,0,8,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-27
1,++/oQ9Lb9dI=,2017,22,0,25,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2017-22
2,++14g8obpj0=,2016,36,0,15,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-36
3,++14g8obpj0=,2016,35,1,18,0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-35
4,++14g8obpj0=,2016,31,2,22,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-31
5,++1XMtcwMec=,2016,38,0,5,2,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-38
6,++3gxZFOJCM=,2016,22,0,18,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,30,2016-22
7,++438ugzEhg=,2017,11,0,22,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,45,2017-11
8,++5u+heOZ8o=,2016,43,0,11,2,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-43
9,++70ByX0a3Q=,2016,33,0,22,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,27,2016-33


In [4]:
# Lista de Clientes
clientes = data.customer_tag.unique()
len(clientes)

4016

In [5]:
# Lista de Periodos

def listar_periodos(time_min,time_max):
    year_min = int(time_min[:4])
    year_max = int(time_max[:4])
    week_min = int(time_min[-2:])
    week_max = int(time_max[-2:])
    
    periodos = []

    if year_min == year_max:
        year = year_min
        for week in range(week_min,week_max+1):
            weeks = ""
            if week < 10:
                weeks = "0"+str(week)
            else:
                weeks = str(week)
            periodos.append(str(year)+"-"+weeks)

    else:
        for year in range(year_min,year_max+1):
            if year == year_min:
                for week in range(week_min,53):
                    weeks = ""
                    if week < 10:
                        weeks = "0"+str(week)
                    else:
                        weeks = str(week)
                    periodos.append(str(year)+"-"+weeks)
            elif year == year_max:   
                for week in range(1,week_max+1):
                    weeks = ""
                    if week < 10:
                        weeks = "0"+str(week)
                    else:
                        weeks = str(week)
                    periodos.append(str(year)+"-"+weeks)
            else:    
                for week in range(1,53):
                    weeks = ""
                    if week < 10:
                        weeks = "0"+str(week)
                    else:
                        weeks = str(week)
                    periodos.append(str(year)+"-"+weeks)
    
    return list(periodos)

lista_periodos = listar_periodos(min(data['time']),max(data['time']))

# Generación de la secuencia de perfiles 

Transformamos la secuencia cronológica de comportamientos en símbolos (corpus), definidos por los perfiles de comportamiento y tomando en cuentas las temporadas sin actividad etiquetandolas como "SinActividad"

In [6]:
corpus=[]
    
for index, cliente in enumerate(clientes):
    lista = []
    for index2, periodo in enumerate(lista_periodos):
        lista.append("SinActividad")
        
    corpus.append(lista)   # creamos la lsita de comportamientos en bruto

In [7]:
# cargamos la lista de comportamientos(corpus) con sus valores respectivos.

for i in range(len(data)):
    # en la comportamiendo del cliente_i (extraemos las columnas necesarias)
    fila = data.loc[i,['customer_tag','time','year','week','collective_cluster']]
   
    # en el indice del cliente i_c  y del periodo i_p 
    index_c = list(clientes).index((fila[0]))  # posision de "customer_tag" en la  lista de clientes
    index_p = lista_periodos.index(fila[1])    # posision de "time" en la  lista de periodo  
    
    if i%50000==0:
        print("porsessing: ",i)
    try:
        num = ""
        if int(fila[4]) < 10:
            num = "0" + str(int(fila[4]))
        else:
            num = str(int(fila[4]))
        
        corpus[index_c][index_p] = "cluster_"+num
    except Exception as e:
        print("Oops!  That was no valid index.  Try again...")


porsessing:  0


In [8]:
len(corpus[0]),len(corpus)

(83, 4016)

# Guardamos Corpus Data

In [9]:
df = pd.DataFrame(data={"Clientes": clientes, "Corpus": corpus})
df.head(5)

,Clientes,Corpus
0,+++g8j9k+5A=,"[SinActividad, SinActividad, SinActividad, Sin..."
1,++/oQ9Lb9dI=,"[SinActividad, SinActividad, SinActividad, Sin..."
2,++14g8obpj0=,"[SinActividad, SinActividad, SinActividad, Sin..."
3,++1XMtcwMec=,"[SinActividad, SinActividad, SinActividad, Sin..."
4,++3gxZFOJCM=,"[cluster_30, SinActividad, SinActividad, SinAc..."


In [10]:
path_out='%sCorpus.csv' %(path)
df.to_csv(path_out, sep=',',index=False)